# 1D PeriodicBC, DirecletBC, 

In [12]:
using DiffEqOperators, Plots

nknots = 4
h = 1.0 / (nknots + 1)
knots = range(h, step=h, length=nknots)
ord_deriv = 2
ord_approx = 2

Δ = CenteredDifference(ord_deriv, ord_approx, h, nknots)
bc = Dirichlet0BC(Float64)
bc_p = PeriodicBC(Float64)
bc_n = Neumann0BC(h)

A = Array(Δ * bc_p)[1]
B = Array(Δ * bc)[1]
C = Array(Δ * bc_n)[1]
#heatmap(A)

4×4 Matrix{Float64}:
 -25.0   25.0    0.0    0.0
  25.0  -50.0   25.0    0.0
   0.0   25.0  -50.0   25.0
   0.0    0.0   25.0  -25.0

In [ ]:
using DiffEqOperators, Test

@testset "Auxiliary functions for GhostDerivativeOperator" begin
    # check length and size functions
    L = CenteredDifference(2, 2, 1., 10)
    Q1 = RobinBC((-1.2, .3, .5), (.2, .5, -30.5), 1.)
    Q2 = PeriodicBC(Float64)

    A = L * Q1
    B = L * Q2

    @test ndims(A) == 2
    @test size(A) == (10, 10)
    @test size(A, 1) == 10
    @test length(A) == 100

    @test ndims(B) == 2
    @test size(B) == (10, 10)
    @test size(B, 1) == 10
    @test length(B) == 100
end

# 2D

In [46]:

n = 10

Lxx = CenteredDifference{1}(2,2,1.,n) 
Lyy = CenteredDifference{2}(2,2,1.,n) 
D = Lxx + Lyy

Mxx = Array(D.ops[1], (n,n))
Myy = Array(D.ops[2], (n,n))

D = Lxx + Lyy

Array(D, (n,n))

Mxx


100×120 Matrix{Float64}:
 1.0  -2.0   1.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0  0.0
 0.0   1.0  -2.0   1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   1.0  -2.0   1.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0   1.0  -2.0   1.0      0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0   0.0   1.0  -2.0      0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0   0.0   0.0   1.0  …  -0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0  -0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  -0.0   0.0   0.0  0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0  -0.0   0.0  0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0  -0.0  0.0
 0.0  -0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0  -0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
 0.0   0.0   0.0  -0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  0.0
 ⋮                        

In [17]:

Lxx = CenteredDifference{1}(2,2,1.0,4)
Lyy = CenteredDifference{2}(2,2,1.0,4)
A = Lxx*bc_p + Lyy*bc_p
Array(Lxx*bc_p)[1]


4×4 Matrix{Float64}:
 -2.0   1.0   0.0   1.0
  1.0  -2.0   1.0   0.0
  0.0   1.0  -2.0   1.0
  1.0   0.0   1.0  -2.0

In [45]:
using DiffEqOperators
bc_p = PeriodicBC(Float64)
N = 4
dx = 0.1
Dxx = CenteredDifference(2,2,dx,N)
Dyy = CenteredDifference{2}(2,2,dx,N)
L0 = Dxx+Dyy
bc_px = MultiDimBC{1}(Dirichlet0BC(Float64), (4,4))
bc_py = MultiDimBC{2}(Dirichlet0BC(Float64), (4,4))
bc_co = compose(bc_px,bc_py)
L = Dxx*bc_p + Dyy*bc_p
using Random
L1 = L0*bc_co
Array(Dxx)
size(L)

(4, 4)

In [44]:
# # Poisson Equation
#
# We want to solve the [poisson equation](https://en.wikipedia.org/wiki/Poisson_equation) on the unit interval. It is given by
# `Δu = f` with boundary conditions `u(0) = a` and `u(1) = b`.
# First of all, let us choose some values for the parameters and remark that there is an exact solution:
f = 1.0
a = -1.0
b = 2.0

u_analytic(x) = f / 2 * x^2 + (b - a - f / 2) * x + a

# We would like to recompute this solution numerically
using DiffEqOperators

nknots = 10
h = 1.0 / (nknots + 1)
ord_deriv = 2
ord_approx = 2

Δ = CenteredDifference(ord_deriv, ord_approx, h, nknots)
bc = DirichletBC(a, b)

# Before solving the equation, let's take a look at Δ and bc:
# display(Array(Δ))
# display(bc*zeros(nknots))
# We see that `Δ` is a (lazy) matrix with the Laplace stencil extended over the boundaries.
# And `bc` acts by padding the values just outside the boundaries.

u = (Δ * bc) \ fill(f, nknots)
knots = range(h, step=h, length=nknots)

# Since we used a second order approximation and the analytic solution itself was a second-order
# polynomial, we expect them to be equal up to rounding errors:
using Test
@test u ≈ u_analytic.(knots)


Test Passed
  Expression: u ≈ u_analytic.(knots)
   Evaluated: [-0.7685950413223139, -0.5289256198347106, -0.2809917355371899, -0.024793388429751873, 0.23966942148760348, 0.5123966942148762, 0.7933884297520662, 1.0826446280991737, 1.3801652892561984, 1.6859504132231407] ≈ [-0.768595041322314, -0.5289256198347108, -0.28099173553719015, -0.024793388429751984, 0.23966942148760317, 0.5123966942148759, 0.7933884297520661, 1.0826446280991737, 1.3801652892561984, 1.6859504132231402]

In [41]:
using DiffEqOperators
const bc_p = PeriodicBC(Float64)
N = 4
dx = 0.1
Dxx = CenteredDifference(2,2,dx,N)
Dyy = CenteredDifference{2}(2,2,dx,N)
L0 = Dxx+Dyy
bc_px = MultiDimBC{1}(Dirichlet0BC(Float64), (4,4))
bc_py = MultiDimBC{2}(Dirichlet0BC(Float64), (4,4))
bc_co = compose(bc_px,bc_py)
L1 = L0*bc_co
Array(L1)
Array(L1,4)
Array(L1,(4,4))
using BlockBandedMatrices
BandedBlockBandedMatrix(L1)
BandedBlockBandedMatrix(L1,4)
BandedBlockBandedMatrix(L1,(4,4))

LoadError: cannot declare bc_p constant; it already has a value

In [ ]:
using DiffEqOperators, OrdinaryDiffEq

# # Heat Equation
# This example demonstrates how to combine `OrdinaryDiffEq` with `DiffEqOperators` to solve a time-dependent PDE.
# We consider the heat equation on the unit interval, with Dirichlet boundary conditions:
# ∂ₜu = Δu
# u(x=0,t)  = a
# u(x=1,t)  = b
# u(x, t=0) = u₀(x)
#
# For `a = b = 0` and `u₀(x) = sin(2πx)` a solution is given by:
u_analytic(x, t) = sin(2*π*x) * exp(-t*(2*π)^2)

nknots = 100
h = 1.0/(nknots+1)
knots = range(h, step=h, length=nknots)
ord_deriv = 2
ord_approx = 2

const Δ = CenteredDifference(ord_deriv, ord_approx, h, nknots)
const bc = Dirichlet0BC(Float64)

t0 = 0.0
t1 = 0.03
u0 = u_analytic.(knots, t0)

step(u,p,t) = Δ*bc*u
prob = ODEProblem(step, u0, (t0, t1))
alg = KenCarp4()
sol = solve(prob, alg)

# 3D

In [ ]:
using DiffEqOperators

s = x, y, z = (-5:0.2:5, -5:0.2:5, -5:0.2:5)
dx = dy = dz = x[2] - x[1]

ricker(x::T, y::T, z::T) where T = (4*(x^2+y^2+z^2) - 6)*exp(-(x^2+y^2+z^2))

u0 = [ricker(X, Y, Z) for Z in z, Y in y, X in x]

Dxx = CenteredDifference{1}(2, 4, dx, length(x))
Dyy = CenteredDifference{2}(2, 4, dy, length(y))
Dzz = CenteredDifference{3}(2, 4, dz, length(z))

A = Dxx+Dyy+Dzz
Q = compose(Dirichlet0BC(Float64, length.(s))...)

dt = dx/(sqrt(3)*3e8)
t = 0.0:dt:10/3e8

f(u,p,t) = (3e8)^2 .*(A*Q*u)



function steptime(u,uold,uolder)
    return ((dt^2 .*f(u,0,0) .+ 5u .- 4uold .+ uolder)./2, u, uold)
end
let uolder = deepcopy(u0), uold = deepcopy(u0), u = deepcopy(u0)
    u,uold,uolder = steptime(u0,uold,uolder)
    # @gif for ti in t #4th order time stepper
    #         u, uold, uolder = steptime(u,uold,uolder)
    #         heatmap(u)
    # end
    for ti in t #4th order time stepper
        u, uold, uolder = steptime(u,uold,uolder)
    end
end
